In [2]:
from app.core.config import settings

2024-12-18 15:15:21,359 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_URI
2024-12-18 15:15:21,603 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_USER
2024-12-18 15:15:21,845 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_PASSWORD
2024-12-18 15:15:22,070 - fastapi_project - INFO - Retrieved parameter: /tailorlink/mysql/MYSQL_DB_NAME
2024-12-18 15:15:22,303 - fastapi_project - INFO - Retrieved parameter: /tailorlink/openai/OPENAI_API_KEY
2024-12-18 15:15:22,533 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_URI
2024-12-18 15:15:22,773 - fastapi_project - INFO - Retrieved parameter: /tailorlink/milvus/MILVUS_TOKEN


In [40]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
        """
        당신은 제네시스 차량 매뉴얼에 대한 전문 Q&A 도우미입니다.
        사용자가 질문을 하면 다음 규칙을 따르세요:

        먼저, 사용자가 한 질문이 제네시스 차량과 관련 있는지 판단하세요.

        만약 질문 내용이 제네시스 차량 관련이 아닌 경우(예: 날씨, 음식, 다른 브랜드 차량 등), "제네시스 차량 관련된 질문만 가능합니다." 라고 한 줄로 출력하세요.
        제네시스 차량 관련 질문이더라도, 다른 브랜드(예: 기아, 현대, 벤츠 등)를 언급하거나 비교하는 부분이 있다면 해당 질문은 무효로 간주하고, "제네시스 차량 관련된 질문만 가능합니다." 라고 한 줄로 출력하세요.
        질문이 일반적인 차량 사용에 관련된 질문 이라면 제네시스 판단하세요.
        질문이 순수하게 제네시스 차량과 관련된 경우, 해당 질문이 단순한 하나의 질문인지, 아니면 여러 질문이 복합적으로 포함된 복합질문인지 판단하세요.

        복합질문이라면, 각 서브질문을 명확히 나눈 뒤, 리스트 형태로 출력하세요. (예: ["이 차량은 하이브리드인가요?", "엔진 오일은 어디서 교환하나요?"])
        출력은 조건에 따라 다음 두 가지 형식 중 하나를 따릅니다.

        비관련 혹은 타사 브랜드 언급 시:
        "제네시스 차량 관련된 질문만 가능합니다." (한 줄)
        제네시스 관련 복합질문 시:
        ["첫 번째 하위 질문", "두 번째 하위 질문", ...]
        예를 들어: ["이 차량은 하이브리드인가요?", "엔진 오일은 어디서 교환하나요?"]

        # 입력 질문: {question}
        # Output Format: {format_instructions}
        """
    )
from app.features.manual.models.model import create_openai_model
llm = create_openai_model('gpt-4o-mini')

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from app.features.manual.models.schemas import QuestionList

parser=JsonOutputParser(pydantic_object=QuestionList)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

res = chain.invoke({'question':'차량시동거는법'})
print(res)
res1 =chain.invoke({'question':'티비시청?'})
print(res1)
res2 =chain.invoke({'question':'차량 시동이 안켜지는데 에어컨 때문이야?'})
print(res2)

{'vailid_question': True, 'print_test': '', 'question_list': ['차량 시동 거는 법은 무엇인가요?']}
{'vailid_question': False, 'print_test': '제네시스 차량 관련된 질문만 가능합니다.', 'question_list': []}
{'vailid_question': True, 'print_test': '', 'question_list': ['차량 시동이 안켜지는 이유는 무엇인가요?', '에어컨이 시동에 영향을 미치나요?']}


In [3]:
from app.database.milvus import get_milvus_client
get_milvus_client()

2024-12-18 15:15:25,103 - fastapi_project - INFO - Connected to Milvus at https://in01-8b96d805fab7a0f.aws-ap-northeast-1.vectordb.zillizcloud.com:19533


In [14]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
        """
        사용자의 질문에 대해 차량 매뉴얼 벡터 데이터베이스를 조회하여 답변을 제공하십시오. 답변에는 사용된 참조 문서의 출처를 명확히 적어주세요.

        # Steps

        1. 사용자의 질문을 명확히 이해하고 필요한 정보를 식별합니다.
        2. 차량 매뉴얼의 벡터 데이터베이스를 조회하여 관련 정보를 검색합니다.
        3. 찾은 정보를 바탕으로 사용자 질문에 대한 정확하고 간결한 답변을 작성합니다.
        4. 답변에 사용한 참조 문서의 출처를 꼭 기재합니다.

        # Output Format

        - 사용자 질문에 대한 답변: [응답 내용]
        - 출처: [참조 문서 제목 및 페이지 또는 섹션 번호]

        # Examples

        **입력 예시:**
        "에어필터 교체 방법을 알고 싶어요."

        **출력 예시:**
        - 사용자 질문에 대한 답변: 에어필터는 보통 차량 엔진룸에 위치해 있으며, 커버를 열고 고정 클립을 풀러 교체할 수 있습니다.
        - 출처: 차량 매뉴얼 3장, 5페이지

        # Notes

        - 가능한 한 정확하고 간결하게 정보를 제공하세요.
        - 찾은 정보가 명확하지 않을 경우, 사용자가 이해할 수 있도록 추가 설명을 포함할 수 있습니다.
        - 모든 답변에는 항상 출처를 포함해야 합니다.

        # 입력 질문: {question}
        """
    )
from app.features.manual.models.model import create_openai_model
llm = create_openai_model('gpt-4o-mini')

from app.features.manual.tools.tools import search_milvus


In [2]:
from app.database.milvus import get_milvus_client
get_milvus_client()
from app.features.manual.models.retriever import get_milvus_hybrid_search_retriever

2024-12-18 16:38:11,093 - fastapi_project - INFO - Connected to Milvus at https://in01-8b96d805fab7a0f.aws-ap-northeast-1.vectordb.zillizcloud.com:19533


In [3]:
re = get_milvus_hybrid_search_retriever()

ValidationError: 4 validation errors for MilvusCollectionHybridSearchRetriever
field_embeddings.0.is-instance[Embeddings]
  Input should be an instance of Embeddings [type=is_instance_of, input_value=<function dense_embedding...c at 0x0000027EE3AB87C0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
field_embeddings.0.is-instance[BaseSparseEmbedding]
  Input should be an instance of BaseSparseEmbedding [type=is_instance_of, input_value=<function dense_embedding...c at 0x0000027EE3AB87C0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
field_embeddings.1.is-instance[Embeddings]
  Input should be an instance of Embeddings [type=is_instance_of, input_value=<function sparse_embeddin...c at 0x0000027EE3AB8E00>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
field_embeddings.1.is-instance[BaseSparseEmbedding]
  Input sho

In [5]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

DEVICE = "cpu"

bge_m3_ef = BGEM3EmbeddingFunction(
        model_name='BAAI/bge-m3', # Specify the model name
        device=DEVICE, # Specify the device to use, e.g., 'cpu' or 'cuda:0'
        use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
    )

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 15827.56it/s]


In [ ]:
bge_m3_ef

In [15]:
from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever
from app.database.milvus import get_collection
from pymilvus import WeightedRanker
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

dense_embedding = BGEM3EmbeddingFunction(
    model_path=dense_embedding_model_path,
    embedding_type="dense"
)

sparse_embedding = BGEM3EmbeddingFunction(
    model_path=sparse_embedding_model_path,
    embedding_type="sparse"
)

bge_m3_ef = BGEM3EmbeddingFunction(
        model_name='BAAI/bge-m3', # Specify the model name
        device=DEVICE, # Specify the device to use, e.g., 'cpu' or 'cuda:0'
        use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
    )

def dense_embedding_func(texts: List[str]) -> List[List[float]]:
    # texts를 dense 임베딩으로 변환
    return dense_embedding(texts)

def sparse_embedding_func(texts: List[str]) -> List[List[float]]:
    # texts를 sparse 임베딩으로 변환
    return sparse_embedding(texts)

NameError: name 'BGEM3EmbeddingFunction' is not defined